In [1]:
import os
import pandas as pd

In [2]:
path = 'C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\climate_data'

In [3]:
def load_excel(path):

    arquivos = os.listdir(path)

    # Lista para armazenar os dfs
    df_stacked = None
    colunas = None
    # Loop sobre os arquivos na pasta
    for n, arquivo in enumerate(arquivos):
        # Verifica se o arquivo é um arquivo CSV
        if arquivo.endswith('.CSV'):

            # Constrói o caminho completo para o arquivo
            caminho_arquivo = os.path.join(path, arquivo)
            
            # Leitura do arquivo CSV e pula as primeiras 8 linhas
            df = pd.read_csv(caminho_arquivo, encoding='latin-1', delimiter=';', header=None, skiprows=8)
            filtered_df = df[[0,1,2,7,8,18]]

            if n == 0:
                filtered_df.columns = filtered_df.iloc[0]
                colunas = filtered_df.columns
                filtered_df = filtered_df.drop(index=0)
                df_stacked = filtered_df.copy()
            else:
                filtered_df.columns = colunas
                filtered_df = filtered_df.drop(index=0)
                df_stacked = pd.concat([df_stacked, filtered_df], ignore_index=True)
    
    return df_stacked


In [17]:
df_clima= load_excel(path)

In [28]:
df_clima = df_clima.rename(columns={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':'precipitacao_total',
                                    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':'temp_media_bulbo_seco',
                                    'TEMPERATURA DO PONTO DE ORVALHO (°C)':'temp_media_ponto_orvalho',
                                    'VENTO, VELOCIDADE HORARIA (m/s)':'velocidade_media_vento',})

In [41]:
df_clima[df_clima.columns] = df_clima[df_clima.columns].astype('category')

In [ ]:
def tratar_decimal(coluna):
    df_clima[coluna] = df_clima[coluna].apply(lambda x: x.replace(',', '0.') if x[0] == ',' else x.replace(',', '.'))
    return df_clima

In [44]:
tratar_decimal('velocidade_media_vento')

,Data,Hora UTC,precipitacao_total,temp_media_bulbo_seco,temp_media_ponto_orvalho,velocidade_media_vento
0,2021/01/01,0000 UTC,0.2,"20,8","19,2",1.2
1,2021/01/01,0100 UTC,0,"21,7",19,2.1
2,2021/01/01,0200 UTC,0,"21,6",19,1.5
3,2021/01/01,0300 UTC,0,"22,3","18,4",1.6
4,2021/01/01,0400 UTC,0,"22,5","17,8",1.5
...,...,...,...,...,...,...
105187,2020/12/31,1900 UTC,0,"29,6","16,2",1.2
105188,2020/12/31,2000 UTC,0,"29,2","15,6",0.6
105189,2020/12/31,2100 UTC,0,28,"15,5",1.3
105190,2020/12/31,2200 UTC,0,"27,6","15,2",0.6


In [38]:
df_clima['velocidade_media_vento'][0][0]

'1'

In [35]:
for i in df_clima['velocidade_media_vento']:
    if ',' in i:
        print(i)

1,2
2,1
1,5
1,6
1,5
1,8
1,3
,8
,3
1,2
1,1
1,3
1,4
1,2
1,4
2,3
1,6
3,5
2,3
2,1
2,2
,6
1,3
,9
1,8
,3
,1
2,4
1,8
1,6
,3
1,2
1,4
1,4
1,4
1,1
1,2
,9
1,8
,9
1,3
1,3
1,3
1,7
2,8
2,2
1,7
1,1
2,2
1,8
,2
1,1
,8
,6
1,6
1,5
1,8
1,2
2,2
2,8
1,9
1,1
1,5
1,3
1,5
1,6
1,8
3,8
1,9
1,2
,9
1,3
,6
2,3
,9
1,1
1,4
1,7
1,5
1,7
2,3
1,7
2,1
1,6
1,2
,7
1,2
1,1
,8
,1
,9
1,2
,6
,8
,5
,7
,1
,8
,6
1,8
1,9
,7
1,9
1,7
1,5
1,5
2,4
1,7
2,7
2,7
1,4
2,2
2,1
,8
1,4
1,3
,7
,2
1,6
1,8
1,3
2,5
1,9
1,3
1,4
1,9
1,6
1,5
5,4
,2
,7
,1
1,2
,3
,1
,1
,1
,1
1,2
1,3
1,2
2,4
2,5
1,8
1,3
2,5
2,4
1,4
2,4
,7
,7
,3
,8
,9
2,7
1,6
,7
1,1
,5
,5
,1
,5
1,3
1,8
2,7
2,3
1,9
3,6
1,9
2,3
1,3
1,3
1,8
,1
,3
,5
3,2
,9
,2
,5
,3
,4
,9
,6
,3
1,7
2,1
,7
3,8
,5
1,4
,5
,2
1,7
1,1
,9
,3
,8
2,4
1,3
1,5
,4
,5
,7
1,9
2,5
1,7
2,6
2,6
1,5
2,7
1,8
1,2
,4
1,4
1,9
2,1
1,6
1,3
1,5
1,6
1,2
,6
,3
,8
2,2
2,2
1,8
1,3
1,8
1,8
1,4
1,2
1,3
1,9
2,1
3,4
2,3
2,6
1,8
3,3
,4
,2
,3
1,1
1,4
1,9
2,4
1,8
2,1
1,7
,3
,5
1,2
2,3
1,8
2,6
1,8
1,2
2,1
1,9
1,6
,6
1,6
1,8
2,6
2,3
2,6
2,8
3,3

In [31]:
df_clima[['precipitacao_total', 'temp_media_bulbo_seco', 'temp_media_ponto_orvalho', 'velocidade_media_vento']] = df_clima[['precipitacao_total', 'temp_media_bulbo_seco', 'temp_media_ponto_orvalho', 'velocidade_media_vento']].astype(float)

ValueError: Cannot cast object dtype to float64

In [11]:
df_clima.groupby(['Data']).agg({'precipitacao_total': 'mean',
                                'temp_media_bulbo_seco': 'mean',
                                'temp_media_ponto_orvalho': 'sum',
                                'velocidade_media_vento': 'mean'}).reset_index()

TypeError: agg function failed [how->mean,dtype->object]